## Content-based filters

Recommendations done using content-based recommenders can be seen as a user-specific classification problem. This classifier learns the user's likes and dislikes from the features of the song.

The most straightforward approach is **keyword matching**.

In a few words, the idea behind is to extract meaningful keywords present in a song description a user likes, search for the keywords in other song descriptions to estimate similarities among them, and based on that, recommend those songs to the user.


Because we are working with text and words, **Term Frequency-Inverse Document Frequency (TF-IDF)** can be used for this matching process.
  
We'll go through the steps for generating a **content-based** music recommender system.

In [3]:
import numpy as np
import pandas as pd

In [4]:
from typing import List, Dict

We are going to use TfidfVectorizer from the Scikit-learn package again.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Dataset

We have the [following dataset](https://www.kaggle.com/mousehead/songlyrics/data#). 

This dataset contains name, artist, and lyrics for *57650 songs in English*. The data has been acquired from LyricsFreak through scraping.

In [6]:
songs = pd.read_csv('..\content based recommedation system\songdata.csv')

In [7]:
songs

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...


Because of the dataset being so big, we are going to resample only 5000 random songs.

In [8]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

We can notice also the presence of `\n` in the text, so we are going to remove it.

In [9]:
songs['text'] = songs['text'].str.replace(r'\n', '')

C:\Users\codei\AppData\Local\Temp\ipykernel_36772\269676193.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


After that, we use TF-IDF vectorizerthat calculates the TF-IDF score for each song lyric, word-by-word. 

Here, we pay particular attention to the arguments we can specify.

In [10]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [11]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

For calculating the similarity of one lyric to another, I've used **cosine similarity**.

In [12]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

Once we get the similarities, we'll store in a dictionary the names of the 50  most similar songs for each song in our dataset.

In [13]:
similarities = {}

In [15]:
for i in range(len(cosine_similarities)):
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

After that, we can use that similarity scores to access the most similar items and give a recommendation.

For that, we'll define our Content based recommender class.

In [17]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
            
    def recommend(self, recommendation):
        song = recommendation['song']
        number_songs = recommendation['number_songs']
        recom_song = self.matrix_similar[song][:number_songs]
        self._print_message(song=song, recom_song=recom_song)

In [135]:
#insrecommedations = ContentBasedRecommender(similarities)

Then, we are ready to pick a song from the dataset and make a recommendation.

In [136]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [137]:
recommedations.recommend(recommendation)

The 4 recommended songs for Jehovah And All That Jazz are:
Number 1:
Sing by Glen Campbell with 0.166 similarity score
--------------------
Number 2:
Devil Cried by Black Sabbath with 0.149 similarity score
--------------------
Number 3:
Angelique by Kenny Loggins with 0.141 similarity score
--------------------
Number 4:
Up The Devil's Pay by Old 97's with 0.131 similarity score
--------------------


And we can pick another random song and recommend again:

In [138]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}

In [139]:
recommedations.recommend(recommendation2)

The 4 recommended songs for I Do It For Your Love are:
Number 1:
I Love You by Lionel Richie with 0.189 similarity score
--------------------
Number 2:
Just One Love by Michael Bolton with 0.187 similarity score
--------------------
Number 3:
I'm Gonna Sit Right Down And Write Myself A Letter by Nat King Cole with 0.184 similarity score
--------------------
Number 4:
If I Love Again by Barbra Streisand with 0.183 similarity score
--------------------
